In [2]:
!pip install geopandas

     -------------------------------------- 338.4/338.4 kB 7.0 MB/s eta 0:00:00
     --------------------------------------- 22.9/22.9 MB 17.7 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 13.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install rasterstats

     ---------------------------------------- 24.5/24.5 MB 9.2 MB/s eta 0:00:00
     ---------------------------------------- 75.9/75.9 kB ? eta 0:00:00



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
###importing all the needed packagesfor this project
import rasterio
import rasterio.mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas as gpd
import numpy as np
import pandas as pd
from rasterstats import zonal_stats


In [6]:
import geemap, ee

In [7]:
import ee
ee.Authenticate()
#### connecting to my EEG cloud project account
ee.Initialize(project='emmanuella-gee-project')

In [15]:
###### we are using the year 2020
year = 2020

In [17]:
#### lets load the regions in Ghana
gaul_level1 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
ghana_regions = gaul_level1.filter(ee.Filter.eq("ADM0_NAME", "Ghana"))
ghana_geom = ghana_regions.geometry()

In [18]:
print("Number of regions in Ghana:", ghana_regions.size().getInfo())


Number of regions in Ghana: 10


In [19]:
###nighttime light (VIIRS)
start = ee.Date.fromYMD(year, 1, 1)
end   = ee.Date.fromYMD(year, 12, 31)


In [20]:
ntl_ghana = (
    ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")
    .filterDate(start, end)
    .select("avg_rad")
    .mean()
    .rename("ntl")
    .clip(ghana_geom)
)


In [21]:
# VISUALIZATION SETTINGS FOR NTL
ntl_vis = {
    "min": 0,
    "max": 30,
    "palette": ["000000", "0d0887", "6a00a8", "b12a90", "e16462", "fca636", "f0f921"]
}

In [46]:
# MAP 1: NIGHTTIME LIGHTS
map1 = geemap.Map(center=[7.9, -1.0], zoom=7)  # roughly center of Ghana
map1.addLayer(ntl_ghana, ntl_vis, "Nighttime Lights (Ghana)")
map1.addLayer(ghana_regions.style(color="white", fillColor="00000000", width=1), {}, "Regions")
map1.add_layer_control()
map1

Map(center=[7.9, -1.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…

In [47]:
###cleaning the map
map1_1 = geemap.Map(center=[7.9, -1.0], zoom=6)
map1_1.addLayer(ntl_ghana.mask(ntl_ghana), name= 'DNB NTL 2020', opacity=0.75)

In [48]:
map1_1.addLayerControl()
map1_1

Map(center=[7.9, -1.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…

In [26]:
########### now unto population for Ghana
#### using Worldpop
##### ISO3 for Ghana = "GHA"
pop_ghana = (
    ee.ImageCollection("WorldPop/GP/100m/pop")
    .filter(ee.Filter.eq("country", "GHA"))
    .filter(ee.Filter.eq("year", year))
    .select("population")
    .first()
    .rename("pop")
    .clip(ghana_geom)
)

In [27]:
########## visuals settings for population
pop_vis = {
    "min": 0,
    "max": 500,
    "palette": ["ffffcc", "a1dab4", "41b6c4", "2c7fb8", "253494"]
}

In [29]:
#### making the second map (population)
map2 = geemap.Map(center=[7.9, -1.0], zoom=7)
map2.addLayer(pop_ghana, pop_vis, "Population (Ghana)")
map2.addLayer(ghana_regions.style(color="black", fillColor="00000000", width=1), {}, "Regions")
map2.add_layer_control()
map2

Map(center=[7.9, -1.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…

In [30]:
# ------------ 4. LIGHT PER CAPITA (LPC) ------------
lpc_ghana = ntl_ghana.divide(pop_ghana.add(0.001)).rename("lpc")

# VIS SETTINGS FOR LPC (you may tweak max)
lpc_vis = {
    "min": 0,
    "max": 0.5,
    "palette": ["440154", "3b528b", "21918c", "5ec962", "fde725"]
}


In [31]:
# MAP 3: LIGHT PER CAPITA
map3 = geemap.Map(center=[7.9, -1.0], zoom=6)
map3.addLayer(lpc_ghana, lpc_vis, "Light per Capita (Ghana)")
map3.addLayer(ghana_regions.style(color="white", fillColor="00000000", width=1), {}, "Regions")
map3.add_layer_control()
map3

Map(center=[7.9, -1.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…

In [ ]:
### Map interpretation

### 1. Strong LPC concentrations in southern Ghana (urban + industrial zones): Southern Ghana shows multiple concentrated high-LPC pockets:
#Accra Metropolitan
#Tema industrial belt
#Koforidua–Suhum corridor
#Kumasi urban center
#Takoradi–Sekondi + Western industrial areas
###These are areas where:Electricity access is highest, Commercial and service-sector activities cluster, Night-time economy is active (transport, malls, service industries), Street lighting and dense settlement raise NTL values
###Economic interpretation: These zones host: High-income earners, Dense commercial markets, Manufacturing hubs, Ports and continuous 24-hour activity (Tema, Takoradi). They likely contribute disproportionately to Ghana’s GDP relative to their population.

###2. Moderate LPC in peri-urban zones. The areas around:
#Eastern Region (Nsawam, Suhum)
#Ashanti fringes (Ejisu, Abuakwa)
#Central Region around Kasoa
#Sunyani peri-urban belt
#show moderate LPC values.
##Interpretation: These are transition zones where: Urban sprawl increases electricity consumption, Light infrastructure (streetlights, shops, highways) expands. But population growth is also high → dilutes LPC. They reflect emerging economic corridors.

###3. Very low LPC in Northern Ghana: The regions north of the transition zone display extremely low light per capita:
#Northern Region
#Savannah Region
#North East
#Upper East & Upper West
##Interpretation: These patterns strongly correspond to: Lower electrification rates,Lower industrialization, Sparse commercial activity, Rural settlements with minimal lighting, Agriculture-dominant livelihoods with low night activity
#Low LPC here signals: Energy poverty, Underdeveloped infrastructure, Low night-time economic output, Limited industrial or service-sector presence. This finding aligns with national electrification and poverty data.

###4. High-LPC “hotspots” in unexpected rural areas:Some small, isolated yellow patches appear in rural zones.

#Interpretation (Possible causes): Mining towns with intensive lighting, Small but wealthy enclaves (private estates / grid expansions), Oil or gas-flaring pixels, Industrial facilities, Highway intersections with strong lighting
#These can or should be cross-checked with: OpenStreetMap industrial layers,Ghana Minerals Commission data, Road network maps for data accuracy and correct interpretation

In [ ]:
##### Also, Testing for poverty hotspots (taking into consideration high population, low lights)
# condition: High population (>200) AND low light (<1)
poverty_hotspots_ghana = pop_ghana.gt(200).And(ntl_ghana.lt(1)).selfMask().rename("poverty_hotspot")

poverty_vis = {
    "min": 0,
    "max": 1,
    "palette": ["ff0000"]   # red
}


In [63]:
# MAP 4: POVERTY HOTSPOTS
map4 = geemap.Map(center=[7.9, -1.0], zoom=6)
map4.addLayer(poverty_hotspots_ghana, poverty_vis, "High Pop / Low Light (Hotspots)")
map4.addLayer(ghana_regions.style(color="orange", fillColor="red", width=2), {}, "Regions")
map4.add_layer_control()
map4

Map(center=[7.9, -1.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…

In [66]:
#########Zonal Statistics, View Result Table
stack_ghana = ntl_ghana.addBands(pop_ghana).addBands(lpc_ghana)


In [67]:
##### ZONAL STATS (MEAN + SUM) ###
reducer = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.sum(),
    sharedInputs=True
)

ghana_stats = stack_ghana.reduceRegions(
    collection=ghana_regions,
    reducer=reducer,
    scale=500
)

In [68]:
# Add country/year fields
ghana_stats = ghana_stats.map(
    lambda f: f.set({
        "country": "Ghana",
        "year": year
    })
)

In [70]:
#############PREVIEW RESULTS AS TABLE (PANDAS)
# This pulls the feature collection to a pandas dataframe (small – okay)
df_ghana = geemap.ee_to_pandas(ghana_stats)

AttributeError: module 'geemap' has no attribute 'ee_to_pandas'

In [ ]:
#############################################lets do for Nigeria also###########################

In [36]:
#### lets load the regions in Ghana
gaul_level1 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
nigeria_states = gaul_level1.filter(ee.Filter.eq("ADM0_NAME", "Nigeria"))
nigeria_geom = nigeria_states.geometry()

In [37]:
print("Number of states in Nigeria:", nigeria_states.size().getInfo())

Number of states in Nigeria: 37


In [38]:
##NIGHTTIME LIGHTS (VIIRS)
start = ee.Date.fromYMD(year, 1, 1)
end   = ee.Date.fromYMD(year, 12, 31)

ntl_nigeria = (
    ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")
    .filterDate(start, end)
    .select("avg_rad")
    .mean()
    .rename("ntl")
    .clip(nigeria_geom)
)


In [39]:
# VISUALIZATION SETTINGS FOR NTL
ntl_vis = {
    "min": 0,
    "max": 30,
    "palette": ["000000", "0d0887", "6a00a8", "b12a90", "e16462", "fca636", "f0f921"]
}

In [40]:
# mapping NIGHTTIME LIGHTS for nigeria
map2_1 = geemap.Map(center=[7.9, -1.0], zoom=6)  # roughly center of nigeria
map2_1.addLayer(ntl_nigeria, ntl_vis, "Nighttime Lights (Nigeria)")
map2_1.addLayer(nigeria_states.style(color="white", fillColor="00000000", width=1), {}, "States")
map2_1.add_layer_control()
map2_1

Map(center=[7.9, -1.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…

In [43]:
###cleaning the map
map2_2 = geemap.Map(center=[7.9, -1.0], zoom=6)
map2_2.addLayer(ntl_nigeria.mask(ntl_nigeria), name= 'DNB NTL 2020', opacity=0.75)

In [44]:
map2_2.addLayerControl()
map2_2

Map(center=[7.9, -1.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…

In [ ]:
# POPULATION (WORLDPOP) ################
# ISO3 for Nigeria = "NGA"
pop_nigeria = (
    ee.ImageCollection("WorldPop/GP/100m/pop")
    .filter(ee.Filter.eq("country", "NGA"))
    .filter(ee.Filter.eq("year", year))
    .select("population")
    .first()
    .rename("pop")
    .clip(nigeria_geom)
)


In [50]:
# VIS SETTINGS FOR POPULATION (log scale-like)
pop_vis = {
    "min": 0,
    "max": 500,
    "palette": ["ffffcc", "a1dab4", "41b6c4", "2c7fb8", "253494"]
}

In [52]:
# MAP 2: POPULATION
map2_3 = geemap.Map(center=[7.9, -1.0], zoom=6)
map2_3.addLayer(pop_nigeria, pop_vis, "Population (Nigeria)")
map2_3.addLayer(nigeria_states.style(color="black", fillColor="00000000", width=1), {}, "States")
map2_3.add_layer_control()
map2_3

Map(center=[7.9, -1.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…

In [54]:
###################Light per Capita Map
lpc_nigeria = ntl_nigeria.divide(pop_nigeria.add(0.001)).rename("lpc")

In [56]:
# VIS SETTINGS FOR LPC for nigeria
lpc_vis = {
    "min": 0,
    "max": 0.7,
    "palette": ["440154", "3b528b", "21918c", "5ec962", "fde725"]
}

In [57]:
# MAP 3: LIGHT PER CAPITA
map2_4 = geemap.Map(center=[7.9, -1.0], zoom=6)
map2_4.addLayer(lpc_nigeria, lpc_vis, "Light per Capita (Nigeria)")
map2_4.addLayer(nigeria_states.style(color="white", fillColor="00000000", width=1), {}, "States")
map2_4.add_layer_control()
map2_4

Map(center=[7.9, -1.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…

In [ ]:
####### Interpretation

In [58]:
######## POVERTY HOTSPOTS ######
# High population (>200) AND low light (<1)
poverty_hotspots_nigeria = pop_nigeria.gt(200).And(ntl_nigeria.lt(1)).selfMask().rename("poverty_hotspot")

poverty_vis = {
    "min": 0,
    "max": 1,
    "palette": ["ff0000"]   # red
}

In [65]:
# POVERTY HOTSPOTS
map2_5 = geemap.Map(center=[7.9, -1.0], zoom=6)
map2_5.addLayer(poverty_hotspots_nigeria, poverty_vis, "High Pop / Low Light (Hotspots)")
map2_5.addLayer(nigeria_states.style(color="orange", fillColor="red", width=1), {}, "States")
map2_5.add_layer_control()
map2_5

Map(center=[7.9, -1.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…